In [6]:


import cv2 as cv 
from mtcnn import MTCNN
import threading
import uuid
import time 



<br>

---
---

# 1. `Record Video:`

---
---

<br>


In [2]:

# ---------------------------- Collecting the user Video ------------------------

fourcc = cv.VideoWriter_fourcc(*"XVID")
writer = cv.VideoWriter("test_video/data_collect.mp4", fourcc, 15.0, (780, 480))
cap = cv.VideoCapture(0)
cnt = 0

while cap.isOpened():
    ret, frame = cap.read()
    frame = cv.flip(frame,1)
    cnt+=1
    if not ret:
        break
    
    # Resize frame if needed to match the VideoWriter's frameSize
    frame = cv.resize(frame, (780, 480))
    writer.write(frame)

    if cnt>0 and cnt<70:
        cv.putText(frame,"Look Forward",(50,250),fontFace=cv.FONT_HERSHEY_SIMPLEX,
                   fontScale=1,color=(0,255,0),
                   thickness=5)
    
    if cnt>70 and cnt<100:
        cv.putText(frame,"Open Your Mouth",(50,250),fontFace=cv.FONT_HERSHEY_SIMPLEX,
                   fontScale=1,color=(0,255,0),
                   thickness=5)
    if cnt>100 and cnt<130:
        cv.putText(frame,"Blink Your Eyes",(50,250),fontFace=cv.FONT_HERSHEY_SIMPLEX,
                   fontScale=1,color=(0,255,0),
                   thickness=5)
    if cnt>130 and cnt<160:
        cv.putText(frame,"Look Left",(50,250),fontFace=cv.FONT_HERSHEY_SIMPLEX,
                   fontScale=1,color=(0,255,0),
                   thickness=5)
    if cnt>160 and cnt<190:
        cv.putText(frame,"Look right",(50,250),fontFace=cv.FONT_HERSHEY_SIMPLEX,
                   fontScale=1,color=(0,255,0),
                   thickness=5)
        
    if cnt>190 and cnt<220:
        cv.putText(frame,"Look Down",(50,250),fontFace=cv.FONT_HERSHEY_SIMPLEX,
                   fontScale=1,color=(0,255,0),
                   thickness=5)
    

    cv.imshow("COLLECT_DATA", frame)

    # Break loop on 'q' key press
    if cv.waitKey(1) & 0xFF == ord('q') or cnt==220:
        break

# Release resources
cap.release()
writer.release()
cv.destroyAllWindows()


OpenCV: FFMPEG: tag 0x44495658/'XVID' is not supported with codec id 12 and format 'mp4 / MP4 (MPEG-4 Part 14)'
OpenCV: FFMPEG: fallback to use tag 0x7634706d/'mp4v'
qt.qpa.plugin: Could not find the Qt platform plugin "wayland" in "/run/media/hadi/5f8d8692-3a2d-40bb-80e6-ad56a68d2e5c/3rd_Semester/3rd_sem_arduino_project/venv/lib/python3.12/site-packages/cv2/qt/plugins"


<br>

---
---
<br>

# `Collect Image With MTCNN`

<br>

---
---
<br>

In [7]:

# ------------------- collect image from the video ------------------------


# Initialize MTCNN detector
detector = MTCNN()

# Directory to save the images
DIR_NAME = "sojib"
# Path to the video file
path = "test_video/data_collect.mp4"

# Open the video file
cap = cv.VideoCapture(path)

# Function to perform face detection and draw rectangles
def detect_and_draw(frame):
    output = detector.detect_faces(frame)
    x, y, width, height = output[0]['box']
    cv.rectangle(img=frame, pt1=(x, y), pt2=(x + width, y + height), color=(255, 0, 0), thickness=5)
    modified_frame = frame[y:y+height, x:x+width]
    return frame, modified_frame

# Initialize variables for timing
last_processed_time = time.time()
processing_interval = 1  # Process every second

# Lock for thread-safe operations
lock = threading.Lock()
detected_frame = None  # Variable to hold the frame with detected faces
processing_frame = None
modified_frame = None  # Frame currently being processed by the thread

# Thread function for detection
def detection_thread():
    global processing_frame, detected_frame,modified_frame
    while cap.isOpened():
        if processing_frame is not None:
            with lock:
                # Run face detection and update the detected_frame
                detected_frame,modified_frame = detect_and_draw(processing_frame)
                processing_frame = None  # Reset processing frame once done

# Start the detection thread
thread = threading.Thread(target=detection_thread, daemon=True)
thread.start()

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break
    
    # Get the current time
    current_time = time.time()
    
    # Check if 1 second has passed and no frame is currently being processed
    if current_time - last_processed_time >= processing_interval and processing_frame is None:
        last_processed_time = current_time
        with lock:
            processing_frame = frame.copy() 

    # If face detection was done, use the detected frame, otherwise use the original frame
    with lock:
        if detected_frame is not None:
            modified_frame, display_frame = modified_frame, detected_frame
        else:
            display_frame = frame

    # BGR to RGB, flip and show display
    display_frame = cv.cvtColor(display_frame, cv.COLOR_BGR2RGB)
    display_frame = cv.flip(display_frame, 1)

    # Save the modified frame
    if modified_frame is not None:
        unique_output_frame_name = "{}/{}.jpg".format(DIR_NAME, uuid.uuid1())
        cv.imwrite(unique_output_frame_name, modified_frame)

    # Show the output
    cv.imshow('output', display_frame)
    
    if cv.waitKey(1) == ord('q'):
        break

# Release the capture and close windows
cap.release()
cv.destroyAllWindows()

